# Interactive Jupyter Notebook

You can experiment with the example code we used in the previous section in this interactive notebook.
Use the launcher button above to open this notebook in Binder or Google Colab.
Please uncomment the line below if you use the Google Colab. (It does not include these packages by default).

In [ ]:
#%pip install pyneuroml neuromllite NEURON

In [1]:
from neuroml import NeuroMLDocument
from neuroml import Izhikevich2007Cell
from neuroml import Network
from neuroml import ExpOneSynapse
from neuroml import Population
from neuroml import PulseGenerator
from neuroml import ExplicitInput
from neuroml import SynapticConnection
import neuroml.writers as writers
import random
from neuroml.utils import validate_neuroml2
from pyneuroml import pynml
from pyneuroml.lems import LEMSSimulation


nml_doc = NeuroMLDocument(id="IzNet")

iz0 = Izhikevich2007Cell(
    id="iz2007RS0", v0="-60mV", C="100pF", k="0.7nS_per_mV", vr="-60mV",
    vt="-40mV", vpeak="35mV", a="0.03per_ms", b="-2nS", c="-50.0mV", d="100pA")
nml_doc.izhikevich2007_cells.append(iz0)

syn0 = ExpOneSynapse(id="syn0", gbase="65nS", erev="0mV", tau_decay="3ms")
nml_doc.exp_one_synapses.append(syn0)

net = Network(id="IzNet")
nml_doc.networks.append(net)

size0 = 5
pop0 = Population(id="IzPop0", component=iz0.id, size=size0)
net.populations.append(pop0)

size1 = 5
pop1 = Population(id="IzPop1", component=iz0.id, size=size1)
net.populations.append(pop1)

random.seed(42)
prob_connection = 0.5
for pre in range(0, size0):
    pg = PulseGenerator(
        id="pulseGen_%i" % pre, delay="0ms", duration="10000ms",
        amplitude="%f nA" % (0.1 * random.random())
    )
    nml_doc.pulse_generators.append(pg)

    exp_input = ExplicitInput(target="%s[%i]" % (pop0.id, pre), input=pg.id)
    net.explicit_inputs.append(exp_input)

    for post in range(0, size1):
        # 'from_' is used since 'from' is Python keyword
        if random.random() <= prob_connection:
            syn = SynapticConnection(from_="%s[%i]" % (pop0.id, pre),
                                     synapse=syn0.id,
                                     to="%s[%i]" % (pop1.id, post))
            net.synaptic_connections.append(syn)

nml_file = 'izhikevich2007_network.nml'
writers.NeuroMLWriter.write(nml_doc, nml_file)

print("Written network file to: " + nml_file)
validate_neuroml2(nml_file)

simulation_id = "example-izhikevich2007cell-sim"
simulation = LEMSSimulation(sim_id=simulation_id,
                            duration=10000, dt=0.1, simulation_seed=123)
simulation.assign_simulation_target(net.id)
simulation.include_neuroml2_file(nml_file)

simulation.create_event_output_file(
    "pop0", "%s.spikes.dat" % simulation_id, format='ID_TIME'
)

for pre in range(0, size0):
    simulation.add_selection_to_event_output_file(
        "pop0", pre, 'IzPop0[{}]'.format(pre), 'spike')

lems_simulation_file = simulation.save_to_file()

pynml.run_lems_with_jneuroml_neuron(
    lems_simulation_file, max_memory="20G", nogui=True, plot=False
)


Written network file to: izhikevich2007_network.nml
Validating izhikevich2007_network.nml against /usr/lib/python3.9/site-packages/neuroml/nml/NeuroML_v2.1.xsd
It's valid!
pyNeuroML >>> Written LEMS Simulation example-izhikevich2007cell-sim to file: LEMS_example-izhikevich2007cell-sim.xml
pyNeuroML >>> *** Problem running command: 
pyNeuroML >>>        Command 'java -Xmx20G  -Djava.awt.headless=true -jar  "/usr/share/java/jNeuroML-0.10.0.jar"  "LEMS_example-izhikevich2007cell-sim.xml"  -neuron -run -compile -nogui' returned non-zero exit status 1.
pyNeuroML >>>  jNeuroML >>   jNeuroML v0.10.0
pyNeuroML >>>  jNeuroML >>  (INFO) Reading from: /home/asinha/Documents/02_Code/00_mine/2020-OSB/NeuroML-Documentation/source/Userdocs/LEMS_example-izhikevich2007cell-sim.xml
pyNeuroML >>>  jNeuroML >>  (INFO) Creating NeuronWriter to output files to /home/asinha/Documents/02_Code/00_mine/2020-OSB/NeuroML-Documentation/source/Userdocs
pyNeuroML >>>  jNeuroML >>  (INFO) Adding simulation Component(

SystemExit: -1

/usr/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3425: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
